In [1]:
import numpy as np
import copy
import open3d as o3d

In [2]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])



In [3]:
print("1. Load two point clouds and show initial pose")
source = o3d.io.read_point_cloud("1.ply")
target = o3d.io.read_point_cloud("2.ply")
# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result_original_color(source, target,
                                        current_transformation)

1. Load two point clouds and show initial pose


In [4]:
# point to plane ICP
current_transformation = np.identity(4)
print("2. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.02.")
result_icp = o3d.registration.registration_icp(
        source, target, 0.02, current_transformation,
        o3d.registration.TransformationEstimationPointToPlane())    
print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation)

2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.


In [5]:
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]

current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                relative_rmse=1e-6,
                                                max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation)


3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
registration::RegistrationResult with fitness = 0.333430, inlier_rmse = 0.018162, and correspondence_set size of 2299
Access transformation to get result.
[30, 0.02, 1]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
registration::RegistrationResult with fitness = 0.424918, inlier_rmse = 0.012810, and correspondence_set size of 5450
Access transformation to get result.
[14, 0.01, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
registration::RegistrationResult with fitness = 0.229571, inlier_rmse = 0.006413, and correspondence_set size of 6571
Access transformation to get result.
